# 香るブラック君_Mistral


RTX-4090 実行: 2024/9/14

In [1]:
!nvidia-smi

Sun Sep 15 00:19:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 546.80       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090 L...    On | 00000000:01:00.0  On |                  N/A |
| N/A   48C    P8                4W / 148W|    110MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%cd ../
%cd SadTalker
# %ls

/home/atsu/llamaindex
/home/atsu/llamaindex/SadTalker


/home/atsu/anaconda3/envs/llamaindex/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# # Whisperのinstal --> 一度だけ実行
# !pip install git+https://github.com/openai/whisper.git

In [4]:
import uuid # 追加
import re
import os
import glob
import random
import time
import gradio as gr
import io
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import openai
from huggingface_hub import login
import whisper

In [5]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext

使用した画像を削除してしまうので以下のコードをコメントアウトした。  
shutil.move(source_image, input_dir) 

In [6]:
# SadTalker用
# from src.gradio_demo2 import SadTalker
from inference6 import make_video # face_enhancer, background_enhancer 対応

In [7]:
# VOICEVOX 用
from pathlib import Path
import voicevox_core
from voicevox_core import AccelerationMode, AudioQuery, VoicevoxCore
from playsound import playsound
from pydub import AudioSegment
# from IPython.display import Audio

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


In [8]:
# 基本パラメータ
model_id = "tokyotech-llm/Swallow-MS-7b-instruct-v0.1"
peft_name = "../付喪神ジェネレータ/QLoRA_models/black_mistral"

# Gradioを用いたWebアプリ作成

In [9]:
# VOICEVOX によるテキストから音声変換
def VOICEVOX(text, out='output.wav', SPEAKER_ID=2):
    open_jtalk_dict_dir = './open_jtalk_dic_utf_8-1.11'
    acceleration_mode = AccelerationMode.AUTO
    core = VoicevoxCore(
        acceleration_mode=acceleration_mode, open_jtalk_dict_dir=open_jtalk_dict_dir
    )
    core.load_model(SPEAKER_ID)
    audio_query = core.audio_query(text, SPEAKER_ID)
    wav = core.synthesis(audio_query, SPEAKER_ID)
    out_byte = Path(out) # 追加
    out_byte.write_bytes(wav)
    return out

In [10]:
# 音源の速度を変える関数
def sound_speed_change(sound_path, save_path, ratio):
    # 曲の読み込み
    sound = AudioSegment.from_wav(sound_path)
    # 曲のスピードを変更
    sound = sound.speedup(playback_speed=ratio, crossfade=0)
    # 曲を保存する
    sound.export(save_path, format="wav")

In [11]:
# モデルとトークナイザの準備
# 量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
# モデルの設定
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    # token=token, # HuggingFaceにログインしておけば不要
    quantization_config=bnb_config, # 量子化
    device_map='auto',
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2",
)
# tokenizerの設定
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    padding_side="right",
    add_eos_token=True
)
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
%%time
# ファインチューニングモデルの作成
model = PeftModel.from_pretrained(base_model, peft_name)

CPU times: user 2.39 s, sys: 576 ms, total: 2.97 s
Wall time: 1.31 s


In [13]:
# 文章生成関数（EZO用）--> Llama3.1でもこのコードは有効だと分かった
def generate(prompt, max_tokens=256, temperature=1, top_p=0.9, top_k=30):

    prompt = f"""system prompt:あなたは缶コーヒーを飲むユーザーと対話するAIです。大阪弁で対話してください。短く返答してください。
    user prompt: {prompt}"""
    
    messages = [
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.ones(input_ids.shape, dtype=torch.long).to(model.device),
    )
    response = outputs[0][input_ids.shape[-1]:]
    response_text = tokenizer.decode(response, skip_special_tokens=True)
    return response_text


In [14]:
def overall_response(input_mode, input_text, input_audio, image, voice, size, preprocess, face_enhancer, bkgnd_enhancer, 
                     max_tokens, temperature, top_p, top_k):
    
    # GPUメモリをリセット
    torch.cuda.empty_cache()
    
    if input_mode == "Text":
        input_text = input_text
    else:
        input_text = speech_to_text_GitHub(input_audio)

    # GPUメモリをリセット
    torch.cuda.empty_cache()

    # 応答
    code_regex = re.compile('[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠！｀ 　＋￥％abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789]')
    response_text = code_regex.sub('', generate(input_text, max_tokens, temperature, top_p, top_k)).replace("\n","")
        # print(rag_result)
    print("origilal response:", response_text) 

    # 文末の「しゅ」に「。」が入らない場合の修正
    response_text = response_text.replace("ですが","やけど").replace("ありますが","あるんやけど").replace("あり。","ありますねん。").\
                                  replace("しているのやねん。","してるんねん。").replace("わい、思いますねん。","").replace("わい", "俺")

    candidates = ["で、どう思う？", "そうなんや、ええな", "ほんま？", "まじで？", "オチは？"]
    sample = random.choice(candidates)
    response_text = response_text.replace("知らんけど", sample)
    
    print("===============================")
    print("processed response text: " + response_text)

    # GPUメモリをリセット
    torch.cuda.empty_cache()

    # 返答を音声に
    out = "../大阪弁Talker/results/output.wav"
    if voice=="玄野武宏":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=11) #玄野武宏
    elif voice=="白上虎太郎":
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=12) #白上虎太郎
    else:
        response_audio = VOICEVOX(text=response_text, out=out, SPEAKER_ID=13) #青山龍星
    
    print("Sound generation finished.") # ターミナルに表示
    print(response_audio)

    # 画像パスを引数に変更
    image_path = image
    print(image_path)
    
    # 追加のデバッグメッセージ
    print(f"Selected image path: {image_path}")
    if not os.path.isfile(image_path):
        raise ValueError(f"Image file does not exist: {image_path}")
    else:
        print(f"Image file exists: {image_path}")

    result_dir = '../大阪弁Talker/results'
    
    # ファイルパスのデバッグメッセージを追加
    print(f"Pic path before test call: {os.path.join(result_dir, str(uuid.uuid4()), 'input', os.path.basename(image_path))}")

    responce_video = make_video(image_path=image_path,
                                audio_path=response_audio,
                                size=size,
                                preprocess=preprocess,
                                enhancer=face_enhancer,
                                background_enhancer=bkgnd_enhancer,
                                result_dir=result_dir,
                                )
    
    print("Video generation finished.") # ターミナルに表示
    print(responce_video)

    # GPUメモリをリセット
    torch.cuda.empty_cache()

    # return rag_result, responce_video, voice, image_path
    return responce_video, voice, image_path

In [15]:
# Blockの定義
# https://note.com/npaka/n/n2a5112208b8d

# CSSを使用してボタンのスタイルをカスタマイズ
css = """
#custom-button {
    background: #ee7800; /* ボタンの背景色 */
    color: white; /* ボタンのテキスト色 */
    border: none;
    text-align: center;
    font-size: 20px;
    cursor: pointer;
}
"""

with gr.Blocks(css=css) as app:

    # Markdown
    gr.Markdown("""# 超未来の付喪神ジェネレータ　香るブラック君""")

    with gr.Row():
        with gr.Column(scale=1, min_width=300):
            with gr.Row():
                with gr.Column():
                    with gr.Row():
                        with gr.Column():
                            input_mode = gr.Radio(["Text","Voice"], label="Input mode", value="Text")
                            input_text = gr.Textbox(placeholder="Please write the input.", label="input text",)
                            record = gr.Audio(sources=["microphone"], label="Input audio", type="filepath")
                            voice = gr.Dropdown(["玄野武宏", "白上虎太郎", "青山龍星"], label="voice", value="玄野武宏")
                            clear = gr.ClearButton(components=[record])
                        with gr.Column():
                            image = gr.Image(type='filepath') #画像のpathを返す
                            clear = gr.ClearButton(components=[image])
            with gr.Row():
                btn = gr.Button("Response", elem_id="custom-button") #ボタンにIDを指定
            with gr.Row():
                with gr.Column(scale=1):                        
                    with gr.Row():
                        gr.Markdown("LLM congig")
                        max_tokens = gr.Dropdown([32, 64, 128, 256, 512, 1024, 2048, 4096, 8192], label="max tokens", value=64)
                        temperature = gr.Slider(minimum=0.01, maximum=1, value=1, step=0.01, label="temperature", interactive=True)
                        top_p = gr.Slider(minimum=0, maximum=1, value=0.9, step=0.01, label="top_p", interactive=True)
                        top_k = gr.Slider(minimum=1, maximum=100, value=30, step=1, label="top_k", interactive=True)
                with gr.Column(scale=1):
                    gr.Markdown("SadTalker congig")
                    with gr.Row():
                        preprocess = gr.Dropdown(["crop", "extcrop", "resize", "full", "extfull"], label="preprocess", value="extcrop")
                        size = gr.Dropdown([256, 512], label="image size", value=256)
                        face_enhancer = gr.Dropdown([None, "gfpgan", "RestoreFormer"], label="face enhancer", value=None)
                        bkgnd_enhancer = gr.Dropdown([None, "realesrgan"], label="background enhancer", value=None)
            with gr.Row():
                # rag_result = gr.Text(label="RAG result")
                agent_voice = gr.Text(label="Voice of agent")
                agent_image = gr.Text(label="Image of agent")                
        with gr.Column(scale=1, min_width=300):
            response_video = gr.Video(label="Response video")
            # イベントリスナー
            btn.click(fn=overall_response,
                      inputs=[input_mode, input_text, record, image, voice, 
                              size, preprocess, face_enhancer, bkgnd_enhancer, max_tokens, temperature, top_p, top_k],                   
                      outputs=[response_video, agent_voice, agent_image]
                      )

In [ ]:
# Webアプリを起動
app.launch(share=True, debug=True) # share=Trueで一時的に公開される

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://c6f4e83b942e168773.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


origilal response: わい、思いますねん。コーヒーの効果を最大限に引き出すために、空腹時に飲むのが効果的やねん。また、ブラックで飲むか、無脂肪牛乳を入れて砂糖を足すか、脂肪入り牛乳を入れて砂糖とクリームを足すかにも
processed response text: コーヒーの効果を最大限に引き出すために、空腹時に飲むのが効果的やねん。また、ブラックで飲むか、無脂肪牛乳を入れて砂糖を足すか、脂肪入り牛乳を入れて砂糖とクリームを足すかにも
Sound generation finished.
../大阪弁Talker/results/output.wav
/tmp/gradio/cc6231c6af6b55b93fa69750904c60ec0ebcb0bd/B02.png
Selected image path: /tmp/gradio/cc6231c6af6b55b93fa69750904c60ec0ebcb0bd/B02.png
Image file exists: /tmp/gradio/cc6231c6af6b55b93fa69750904c60ec0ebcb0bd/B02.png
Pic path before test call: ../大阪弁Talker/results/f21108f8-9543-450c-a82c-3b350263a724/input/B02.png
using safetensor as default
3DMM Extraction for source image


Face Renderer:: 100%|█████████████████████████████████| 159/159 [00:18<00:00,  8.57it/s]
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (256, 255) to (256, 256) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The generated video is named ../大阪弁Talker/results/2024_09_15_00.20.30/B02##output.mp4
Video generation finished.
../大阪弁Talker/results/2024_09_15_00.20.30.mp4
